In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [2]:
using LinearAlgebra
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using PushRecovery
BLAS.set_num_threads(1)

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.0/PushRecovery/oE0W9.ji for PushRecovery [e5d7e63e-020d-11e9-1e2d-c1b17e981caa]
└ @ Base loading.jl:1190


In [3]:
@time AtlasRobot.mechanism(add_flat_ground=true);

  8.176417 seconds (20.89 M allocations: 1.053 GiB, 7.03% gc time)


In [4]:
@time mechanism = AtlasRobot.mechanism(add_flat_ground=true);

  0.004416 seconds (30.01 k allocations: 1.459 MiB)


In [5]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = point.model.friction.μ
            contact = addcontact!(lowlevel, body, position, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [6]:
# create high level controller
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)
controller = let
    atol_distance = 1e-3
    convexhulloptimizer = OSQP.Optimizer()
    MOI.set(convexhulloptimizer, OSQPSettings.Verbose(), false)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsAbs(), atol_distance^2)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsRel(), 1e-8)
#     MOI.set(convexhulloptimizer, OSQPSettings.Polish(), true)
#     MOI.set(convexhulloptimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    PushRecoveryController(lowlevel, feet, pelvis, nominalstate, convexhulloptimizer)
end;

In [7]:
# create visualizer
using MeshCat
using MeshCatMechanisms

if !(@isdefined gui) || !(@isdefined pushapplier) || !any(isopen, gui.visualizer.visualizer.core.scope.pool.connections)
    pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0));    
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    mvis = MechanismVisualizer(mechanism, visuals)
    gui = GUI(mvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
end

set_configuration!(gui.visualizer, configuration(nominalstate))

In [8]:
# create ODEProblem
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., Inf); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, Inf)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [12]:
# simulate
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
last(sol.t)

 33.426552 seconds (45.89 M allocations: 2.602 GiB, 6.74% gc time)


22.768449193821503

In [10]:
# setanimation!(mvis, sol)

In [26]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.018 ms (0.00% GC)
  median time:      1.119 ms (0.00% GC)
  mean time:        1.113 ms (0.00% GC)
  maximum time:     1.559 ms (0.00% GC)
  --------------
  samples:          4475
  evals/sample:     1